In [1]:
import geopandas as gpd
from sedona.spark import *
from delta import *
from pyspark.sql import SparkSession

packages = [
    'io.delta:delta-core_2.12:2.1.1',
    'org.apache.sedona:sedona-spark-shaded-3.4_2.12:1.5.0,'
    'org.datasyslab:geotools-wrapper:1.5.0-28.2'
]

config = SedonaContext.builder()\
     .config("spark.jars.packages", ",".join(packages))\
     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
     .enableHiveSupport()\
     .getOrCreate()

sedona = SedonaContext.create(config)

gdf = gpd.read_file("taxi_zones.shp")

gpd.GeoDataFrame.iteritems = gpd.GeoDataFrame.items

sedona_df = sedona.createDataFrame(gdf)

sedona_df.write.format('delta').mode('overwrite').saveAsTable('taxi_zones')

In [2]:
%load_ext sparksql_magic

In [3]:
%%sparksql

show databases;

namespace
default


In [4]:
%%sparksql
show tables in default

namespace,tableName,isTemporary
default,device_data_events,False
default,people10m,False
default,taxi_zone_lookup,False
default,taxi_zones,False
